In [ ]:
def ajustar_imagen(ruta, ancho_max, alto_max):
    img = Image(ruta)
    ancho_img, alto_img = img.drawWidth, img.drawHeight

    width_scale = ancho_max / ancho_img
    height_scale = alto_max / alto_img

    scale = max(width_scale, height_scale)

    img.drawWidth = ancho_img * scale
    img.drawHeight = alto_img * scale
    return img

In [ ]:
def recom_obs(info_df, validacion_list, comparacion_metodo_df):
    salida = []

    # SENTIDO
    sentido_definido = not info_df.empty and info_df.get('DESC_SENTIDO').iloc[0] != ""
    sentido_congruente = validacion_list.get('metas') == "Los umbrales y la línea base coinciden con el sentido; " and validacion_list.get('metas') != "Sentido mal determinado"

    if sentido_definido:
        salida.append({"criterio": "Sentido definido", "score": 0.5, "recomendacion": "El indicador tiene un sentido definido.", "observacion": "Información del sentido del indicador registrada."})
    else:
        salida.append({"criterio": "Sentido definido", "score": 0, "recomendacion": "Se recomienda revisar la información registrada.", "observacion": "No se registró sentido"})

    if sentido_congruente:
        salida.append({"criterio": "Congruencia del sentido", "score": 0.5, "recomendacion": "El sentido del indicador es adecuado.", "observacion": ""})
    elif (sentido_definido != sentido_congruente):
         salida.append({"criterio": "Congruencia del sentido", "score": 0.25, "recomendacion": "Se recomienda revisar la congruencia del sentido del indicador con su línea base o con las metas programadas", "observacion": "El sentido del indicador corresponde con un elemento."})
    else:
        salida.append({"criterio": "Congruencia del sentido", "score": 0, "recomendacion": "Se recomienda revisar la congruencia del sentido del indicador con su línea base y con las metas programadas", "observacion": "El sentido del indicador no es congruente con la línea base ni con las metas programadas"})

    # LÍNEA BASE
    linea_base_definida = validacion_list.get('vacios') != 'El ciclo de la línea base no está definido; ' and validacion_list.get('vacios') != 'La línea base no está definida; '
    if linea_base_definida:
        salida.append({"criterio": "Línea base definida", "score": 0.5, "recomendacion": "La línea base y el ciclo de la línea base están definidos.", "observacion": "Línea base y ciclo de la línea base definidos."})
    elif (validacion_list.get('vacios') == 'El ciclo de la línea base no está definido; ') != (validacion_list.get('vacios') == 'La línea base no está definida; '):
        salida.append({"criterio": "Línea base definida", "score": 1/4, "recomendacion": "Se recomienda revisar la información registrada.", "observacion": "Línea base o ciclo línea de la línea base no definido"})
    else:
        salida.append({"criterio": "Línea base definida", "score": 0, "recomendacion": "Se recomienda revisar la información registrada.", "observacion": "No se regitró línea base ni su ciclo."})

    linea_base_congruente = validacion_list.get('umbrales') == "La meta anual coincide con la línea base coinciden con el sentido; "
    indices = validacion_list.get('indices')
    if linea_base_congruente:
        salida.append({"criterio": "Congruencia de la línea base", "score": 0.5, "recomendacion": "La línea base es congruente con las metas y el sentido del indicador.", "observacion": ""})
    elif (validacion_list.get('umbrales') == "La meta anual debe ser menor o igual a la línea base; " or validacion_list.get('umbrales') == "La meta anual debe ser mayor o igual a la línea base; ") or (indices and validacion_list.get('mensajes') == f"META {indices[0]} debe ser mayor o igual que la línea base; ") or (indices and validacion_list.get('mensajes') == f"META {indices[0]} debe ser menor o igual que la línea base; "):
         salida.append({"criterio": "Congruencia de la línea base", "score": 1/4, "recomendacion": "Se recomienda ajustar la línea base para que sea congruente con las metas y sentido del indicador.", "observacion": "La meta anual o la primer meta no son consistentes con el sentido del indicador."})
    else:
        salida.append({"criterio": "Congruencia de la línea base", "score": 0, "recomendacion": "Se recomienda establecer una línea base sólida, con fuente y año claros.", "observacion": "La meta anual y la primer meta no son consistentes con el sentido del indicador."})


    # SEMAFORIZACIÓN
    umbrales_definidos = validacion_list.get('vacios') != "Umbral rojo no definido; " and validacion_list.get('vacios') != "Umbral verde no definido; "
    if umbrales_definidos:
        salida.append({"criterio": "Umbrales definidos", "score": 1/2, "recomendacion": "", "observacion": "Están definidos ambos umbrales"})
    elif (validacion_list.get('vacios') == "Umbral rojo no definido; ") != (validacion_list.get('vacios') == "Umbral verde no definido; "):
        salida.append({"criterio": "Umbrales definidos", "score": 1/4, "recomendacion": "Se recomienda revisar la información registrada.", "observacion": "Uno de los umbrales no está definido."})
    else:
        salida.append({"criterio": "Umbrales definidos", "score": 0, "recomendacion": "Se recomienda definir ambos umbrales.", "observacion": "Umbrales no definidos."})

    umbrales_validos = validacion_list.get('umbrales') == ""
    if umbrales_validos:
        salida.append({"criterio": "Validación umbrales", "score": 1/2, "recomendacion": "Se recomienda validar periódicamente la pertinencia de los rangos definidos.", "observacion": "Los umbrales coinciden con el sentido definido."})
    else:
        salida.append({"criterio": "Validación umbrales", "score": 0, "recomendacion": "Se recomienda definir o rediseñar los parámetros de semaforización con base técnica, metas realistas y criterios claros.", "observacion": "Los umbrales no coinciden con el sentido definido."})

    # FRECUENCIA
    frecuencia_definida = validacion_list.get('vacios') != "Frecuencia no definida; "
    if frecuencia_definida:
        salida.append({"criterio": "Frecuencia definida", "score": 1, "recomendacion": "Frecuencia definida.", "observacion": "Frecuencia de medición del indicador definida."})
    else:
        salida.append({"criterio": "Frecuencia definida", "score": 0, "recomendacion": "Se recomienda definir la frecuencia de medición del indicador.", "observacion": "Frecuencia de medición del indicador no definida."})

    # META ANUAL / PRIMER META
    meta_anual_definida = validacion_list.get('vacios') != 'Meta anual no definida; '
    if meta_anual_definida:
        salida.append({"criterio": "Meta anual definida", "score": 0.5, "recomendacion": "Meta anual definida.", "observacion": "Meta anual definida."})
    else:
        salida.append({"criterio": "Meta anual definida", "score": 0, "recomendacion": "Se recomienda revisar la información registrada.", "observacion": "Meta anual no definida."})

    meta_anual_consistente = not ((validacion_list.get('umbrales') == "La meta anual debe ser menor o igual a la línea base; ") or (validacion_list.get('umbrales') == "La meta anual debe ser mayor o igual a la línea base; ") or (indices and validacion_list.get('mensajes') == f"META {indices[0]} debe ser mayor o igual que la línea base; ") or (indices and validacion_list.get('mensajes') == f"META {indices[0]} debe ser menor o igual que la línea base; "))

    if meta_anual_consistente:
        salida.append({"criterio": "Consistencia de la meta anual", "score": 0.5, "recomendacion": "La meta anual y la primer meta son consistentes con el sentido del indicador.", "observacion": "La meta anual y la primer meta son consistentes con el sentido del indicador."})
    else:
        salida.append({"criterio": "Consistencia de la meta anual", "score": 0, "recomendacion": "La meta anual o la primer meta no son consistentes con el sentido del indicador.", "observacion": "La meta anual o la primer meta no son consistentes con el sentido del indicador."})

    ##METAS CORRESPONDIENTES A LA FRECUENCIA REGSISTRADA
    metas_frecuencia_definidas = validacion_list.get('metas') != "Sin suficientes metas registradas; "
    if metas_frecuencia_definidas:
        salida.append({"criterio": "Metas correspondientes a la frecuencia definida", "score": 0.5, "recomendacion": "Están definidas las metas correspondientes a la frecuencia.", "observacion": "Metas definidas correspondientes a la frecuencia definida."})
    else:
        salida.append({"criterio": "Metas correspondientes a la frecuencia definida", "score": 0, "recomendacion": "Se recomienda revisar las metas registradas o la frecuencia definida.", "observacion": "No se registraron metas correspondientes a la frecuencia definida."})

    metas_frecuencia_consistentes = validacion_list.get('patron_metas') != "Sentido mal determiando"
    if metas_frecuencia_consistentes:
        salida.append({"criterio": "Consistencia de las metas con el sentido", "score": 0.5, "recomendacion": "Las metas son consistentes con el sentido del indicador.", "observacion": "Las metas son consistentes con el sentido del indicador."})
    else:
        salida.append({"criterio": "Consistencia de las metas con el sentido", "score": 0, "recomendacion": "Se recomienda revisar y modificar metas o revisar el sentido y ajustarlo si es necesario.", "observacion": "Las metas no son consistentes con el sentido del indicador."})


    # CONGRUENCIA DE NOMBRE, MÉTODO DE CÁLCULO Y DEFINICIÓN DEL INDICADOR
    parametros_definidos = validacion_list.get('no_info') != "Nombre del indicador no definido; " and validacion_list.get('no_info') != "Definición del indicador no definida; " and validacion_list.get('no_info') != "Método de cálculo del indicador no definido; "
    if parametros_definidos:
        salida.append({"criterio": "Parámetros definidos", "score": 1/3, "recomendacion": "El nombre, definición y método de cálculo del indicador están definidos.", "observacion": "Nombre, definición y método de cálculo del indicador definidos."})
    elif (validacion_list.get('no_info') == "Nombre del indicador no definido; ") != (validacion_list.get('vacios') == "Definición del indicador no definida; ") != (validacion_list.get('vacios') == "Método de cálculo del indicador no definido; "):
         salida.append({"criterio": "Parámetros definidos", "score": 1/6, "recomendacion": "Se recomienda revisar la información registrada.", "observacion": "Nombre, definición o método de cálculo del indicador no definidos."})
    else:
        salida.append({"criterio": "Parámetros definidos", "score": 0, "recomendacion": "Se recomienda definir el nombre, definición y método de cálculo del indicador.", "observacion": "Nombre, definición y método de cálculo del indicador no definidos."})

    nombre_definicion_congruentes = not comparacion_metodo_df.empty and comparacion_metodo_df.get('NOMBRE_DEFINICION_COMPARACION').iloc[0] == "Sí"
    if nombre_definicion_congruentes:
        salida.append({"criterio": "Nombre y definición congruentes", "score": 1/3, "recomendacion": "El nombre y la definición del indicador son congruentes.", "observacion": "Nombre y definición del indicador congruentes."})
    else:
        salida.append({"criterio": "Nombre y definición congruentes", "score": 0, "recomendacion": "Se recomienda revisar la congruencia entre el nombre y la definición del indicador.", "observacion": "Nombre y definición del indicador no son congruentes."})

    definicion_metodo_congruentes = not comparacion_metodo_df.empty and comparacion_metodo_df.get('DEFINICION_METODO_COMPARACION').iloc[0] == "Sí"
    if definicion_metodo_congruentes:
        salida.append({"criterio": "Definición y método de cálculo congruente", "score": 1/3, "recomendacion": "La definición y el método de cálculo del indicador son congruentes.", "observacion": "Definición y método de cálculo del indicador congruentes."})
    else:
        salida.append({"criterio": "Definición y método de cálculo congruente", "score": 0, "recomendacion": "Se recomienda revisar la congruencia entre la definición y el método de cálculo del indicador.", "observacion": "Definición y método de cálculo del indicador no son congruentes."})

    # COMPARACIÓN MÉTODO DE CÁLCULO Y TIPO RELATIVO
    tipo_relativo_metodo_congruente = not comparacion_metodo_df.empty and comparacion_metodo_df.get('TIPO_RELATIVO_METODO_COMPARACION').iloc[0] == "Sí"
    if tipo_relativo_metodo_congruente:
        salida.append({"criterio": "Tipo relativo y método de cálculo congruentes", "score": 1, "recomendacion": "El tipo relativo y el método de cálculo del indicador son congruentes.", "observacion": "Tipo relativo y método de cálculo del indicador congruentes."})
    else:
        salida.append({"criterio": "Tipo relativo y método de cálculo congruentes", "score": 0, "recomendacion": "Se recomienda revisar la congruencia entre el tipo relativo y el método de cálculo del indicador.", "observacion": "Tipo relativo y método de cálculo del indicador no son congruentes."})

    return salida

In [ ]:
def generar_ficha_pdf(ficha_resultado,
                     titulo="Índice de Seguimiento de los Indicadores de Desempeño",
                     nombre_archivo="ficha_indicadores.pdf",
                     ruta=None,
                     imagen_izquierda_path=None,
                     imagen_derecha_path=None):

    if ruta is None:
        ruta = os.getcwd()

    os.makedirs(ruta, exist_ok=True)

    output_path = os.path.join(ruta, nombre_archivo)
    doc = SimpleDocTemplate(output_path, pagesize=letter)
    styles = getSampleStyleSheet()
    #Definir estilos para la tabla
    styles.add(ParagraphStyle(name='WrappedTableText', parent=styles['Normal'], leading=12))

    story = []

    #Imágenes en el encabezado
    header_elements = []
    MAX_IMG_ANCHO = 1.0 * inch
    MAX_IMG_ALTO = 0.75 * inch

    if imagen_izquierda_path and os.path.exists(imagen_izquierda_path):
        img_left = ajustar_imagen(imagen_izquierda_path, MAX_IMG_ANCHO, MAX_IMG_ALTO)
        header_elements.append(img_left)
    else:
        header_elements.append(Paragraph("", styles['Normal']))

    if imagen_derecha_path and os.path.exists(imagen_derecha_path):
        img_right = ajustar_imagen(imagen_derecha_path, MAX_IMG_ANCHO, MAX_IMG_ALTO)
        header_elements.append(img_right)
    else:
        header_elements.append(Paragraph("", styles['Normal']))

    if header_elements:
        header_table = Table([header_elements], colWidths=[letter[0]/2, letter[0]/2])
        header_table.setStyle(TableStyle([
            ('VALIGN', (0,0), (-1,-1), 'TOP'),
            ('ALIGN', (0,0), (0,0), 'LEFT'),
            ('ALIGN', (1,0), (1,0), 'RIGHT'),
            ('LEFTPADDING', (0,0), (-1,-1), 0),
            ('RIGHTPADDING', (0,0), (-1,-1), 0),
            ('TOPPADDING', (0,0), (-1,-1), 0),
            ('BOTTOMPADDING', (0,0), (-1,-1), 0),
        ]))
        story.append(header_table)
        story.append(Spacer(1, 0.2*inch))

    #Para el título
    story.append(Paragraph(titulo, styles['Title']))
    story.append(Spacer(1, 0.2*inch))

    #Información de la ficha de resultado
    info_df = ficha_resultado.get('info', pd.DataFrame())
    no_info_str = ficha_resultado.get('no_info', '')
    validacion_list = ficha_resultado.get('validacion', {})
    corresponde_str = ficha_resultado.get('corresponde', '')
    sentido_str = ficha_resultado.get('sentido', '')
    comparacion_metodo_df = ficha_resultado.get('comparacion_metodo_res', {})

    #INFORMACIÓN BÁSICA
    info_data = [["Seguimiento de los Indicadores"]]
    if not info_df.empty:
        info_data.append(['Ciclo', Paragraph(str(info_df.get('CICLO', '').iloc[0]), styles['WrappedTableText'])])
        info_data.append(['Ramo', Paragraph(str(info_df.get('RAMO', '').iloc[0]), styles['WrappedTableText'])])
        info_data.append(['Clave', Paragraph(str(info_df.get('PP', '').iloc[0]), styles['WrappedTableText'])])
        info_data.append(['Nombre Pp', Paragraph(str(info_df.get('DESC_PROYECTO', '').iloc[0]), styles['WrappedTableText'])])
        info_data.append(['Nivel', Paragraph(str(info_df.get('TIPO_NIVEL', '').iloc[0]), styles['WrappedTableText'])])
        info_data.append(['Indicador', Paragraph(str(info_df.get('KA_INDICADOR', '').iloc[0]), styles['WrappedTableText'])])
        info_data.append(['Nombre del indicador', Paragraph(str(info_df.get('NOMBRE_INDICADOR', '').iloc[0]), styles['WrappedTableText'])])
        info_data.append(['Definición del indicador', Paragraph(str(info_df.get('DEFINICION_INDICADOR', '').iloc[0]), styles['WrappedTableText'])])
        info_data.append(['Método de cálculo', Paragraph(str(info_df.get('METODO_CALCULO_INDICADOR', '').iloc[0]), styles['WrappedTableText'])])
        info_data.append(['Tipo relativo', Paragraph(str(info_df.get('TIPO_RELATIVO', '').iloc[0]), styles['WrappedTableText'])])
        info_data.append(['Sentido', Paragraph(str(info_df.get('DESC_SENTIDO', '').iloc[0]), styles['WrappedTableText'])])
    else:
        info_data.append([Paragraph('No hay datos disponibles para este indicador.', styles['WrappedTableText']), ''])


    info_table = Table(info_data, colWidths=[2*inch, 5*inch]) 
    info_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor("#2A4277")), 
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.HexColor("#EDAB25")),
        ('FONTSIZE', (0, 0), (-1, 0), 16),
        ('ALIGN', (0, 0), (-1, 0), 'CENTRE'),
        ('ALIGN', (0, 1), (0, -1), 'LEFT'), 
        ('ALIGN', (1, 1), (-1, -1), 'LEFT'), 
        ('VALIGN', (0, 0), (-1, -1), 'TOP'), 
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('LEFTPADDING', (0,0), (-1,0), 6),
        ('RIGHTPADDING', (0,0), (-1,0), 6),
        ('TOPPADDING', (0,0), (-1,0), 6),
        ('BOTTOMPADDING', (0,0), (-1,0), 12),
        ('SPAN',(0, 0), (-1, 0)),
    ]))
    story.append(info_table)
    story.append(Spacer(1, 0.2*inch))

    #VALIDACIÓN DE METAS
    story.append(Spacer(1, 0.1*inch))

    validacion_data = [["Validación de Metas"]]
    validacion_data.append(['Campos vacíos', Paragraph(validacion_list.get('vacios', ''), styles['WrappedTableText'])])
    validacion_data.append(['Umbrales', Paragraph(validacion_list.get('umbrales', ''), styles['WrappedTableText'])])
    validacion_data.append(['Metas', Paragraph(validacion_list.get('metas', ''), styles['WrappedTableText'])])
    validacion_data.append(['Sentido determinado', Paragraph(validacion_list.get('patron_metas', ''), styles['WrappedTableText'])])

    validacion_table = Table(validacion_data, colWidths=[2*inch, 5*inch]) 
    validacion_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#2A4277')), 
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
        ('FONTSIZE', (0, 0), (-1, 0), 16),
        ('ALIGN', (0, 0), (-1, 0), 'CENTRE'),
        ('ALIGN', (0, 1), (0, -1), 'LEFT'), 
        ('ALIGN', (1, 1), (-1, -1), 'LEFT'), 
         ('VALIGN', (0, 0), (-1, -1), 'TOP'), 
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('LEFTPADDING', (0,0), (-1,-1), 6),
        ('RIGHTPADDING', (0,0), (-1,-1), 6),
        ('TOPPADDING', (0,0), (-1,-1), 6),
        ('BOTTOMPADDING', (0,0), (-1,-1), 12),
        ('SPAN',(0, 0), (-1, 0)),
    ]))
    story.append(validacion_table)
    story.append(Spacer(1, 0.2*inch))

    #COMPARACIÓN DE MÉTODO DE CÁLCULO, DEFINICIÓN, ETC
    comparacion_data = [["Comparación de Métodos"]]

    comparacion_data.append([Paragraph('Comparación entre nombre y definición', styles['WrappedTableText']), Paragraph(str(comparacion_metodo_df.get('NOMBRE_DEFINICION_COMPARACION', '').iloc[0]) if not comparacion_metodo_df.empty and isinstance(comparacion_metodo_df.get('NOMBRE_DEFINICION_COMPARACION'), pd.Series) else comparacion_metodo_df.get('NOMBRE_DEFINICION_COMPARACION', ''), styles['WrappedTableText'])])
    comparacion_data.append([Paragraph('Comparación entre definición y método de cálculo', styles['WrappedTableText']), Paragraph(str(comparacion_metodo_df.get('DEFINICION_METODO_COMPARACION', '').iloc[0]) if not comparacion_metodo_df.empty and isinstance(comparacion_metodo_df.get('DEFINICION_METODO_COMPARACION'), pd.Series) else comparacion_metodo_df.get('DEFINICION_METODO_COMPARACION', ''), styles['WrappedTableText'])])
    comparacion_data.append([Paragraph('Comparación entre tipo relativo y método de cálculo registrado', styles['WrappedTableText']), Paragraph(str(comparacion_metodo_df.get('TIPO_RELATIVO_METODO_COMPARACION', '').iloc[0]) if not comparacion_metodo_df.empty and isinstance(comparacion_metodo_df.get('TIPO_RELATIVO_METODO_COMPARACION'), pd.Series) else comparacion_metodo_df.get('TIPO_RELATIVO_METODO_COMPARACION', ''), styles['WrappedTableText'])])
    comparacion_data.append([Paragraph('Comparación con otro tipo relativo', styles['WrappedTableText']), comparacion_metodo_df.get('OTRA_COINCIDENCIA', '').iloc[0] if not comparacion_metodo_df.empty and isinstance(comparacion_metodo_df.get('OTRA_COINCIDENCIA'), pd.Series) else comparacion_metodo_df.get('OTRA_COINCIDENCIA', '')])

    comparacion_table = Table(comparacion_data, colWidths=[2*inch, 5*inch])
    comparacion_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#2A4277')), 
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
        ('FONTSIZE', (0, 0), (-1, 0), 16),
        ('ALIGN', (0, 0), (-1, 0), 'CENTRE'),
        ('ALIGN', (0, 1), (0, -1), 'LEFT'), 
        ('ALIGN', (1, 1), (-1, -1), 'LEFT'), 
         ('VALIGN', (0, 0), (-1, -1), 'TOP'), 
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('LEFTPADDING', (0,0), (-1,-1), 6),
        ('RIGHTPADDING', (0,0), (-1,-1), 6),
        ('TOPPADDING', (0,0), (-1,-1), 6),
        ('BOTTOMPADDING', (0,0), (-1,-1), 12),
        ('SPAN',(0, 0), (-1, 0)),
    ]))
    story.append(comparacion_table)
    story.append(Spacer(1, 0.2*inch))

    #OBSERVACIONES Y PUNTAJE
    validation_results = recom_obs(info_df, validacion_list, comparacion_metodo_df)
    score_data = [["Criterios", "Puntaje", ""]]

    for result in validation_results:
        criterio = result["criterio"]
        score_value = result["score"]  
        recomendacion = result["recomendacion"]
        observacion = result["observacion"]

        diameter = 0.2 * inch
        radius = diameter / 2.0
        circle_drawing = Drawing(diameter, diameter)
        circle_color = colors.grey 

        if score_value > 1/4:
            circle_color = colors.HexColor('#4CAF50')  
        elif score_value > 0:
            circle_color = colors.HexColor('#FFEB3B') 
        else:
            circle_color = colors.HexColor('#F44336') 

        circle = Circle(radius, radius, radius)
        circle.fillColor = circle_color
        circle.strokeColor = circle_color
        circle_drawing.add(circle)

        #Para mostrar el cirterio, la observación y el semáforo correspondiente
        display_text = f"<b>{criterio}:</b> {observacion}"
        score_data.append([Paragraph(display_text, styles['WrappedTableText']), f"{score_value:.2f}", circle_drawing])


    score_table = Table(score_data, colWidths=[5*inch, 1*inch, 1*inch])
    score_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#2A4277')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
        ('FONTSIZE', (0, 0), (-1, 0), 16),
        ('ALIGN', (0, 0), (-1, 0), 'CENTRE'),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('LEFTPADDING', (0,0), (-1,-1), 6),
        ('RIGHTPADDING', (0,0), (-1,-1), 6),
        ('TOPPADDING', (0,0), (-1,-1), 6),
        ('BOTTOMPADDING', (0,0), (-1,-1), 12),
    ]))
    story.append(score_table)
    story.append(Spacer(1, 0.2*inch))

    #RECOMENDACION
    oportunidad_data = [["Recomendaciones"]]
    for result in validation_results:
        recomendacion = result["recomendacion"]
        display_text = f"<b>{result['criterio']}:</b> {recomendacion}"
        oportunidad_data.append([Paragraph(display_text, styles['WrappedTableText'])])

    oportunidad_table = Table(oportunidad_data, colWidths=[7*inch]) 
    oportunidad_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#2A4277')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
        ('FONTSIZE', (0, 0), (-1, 0), 16),
        ('ALIGN', (0, 0), (-1, 0), 'CENTRE'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('LEFTPADDING', (0,0), (-1,-1), 6),
        ('RIGHTPADDING', (0,0), (-1,-1), 6),
        ('TOPPADDING', (0,0), (-1,-1), 6),
        ('BOTTOMPADDING', (0,0), (-1,-1), 12),
        ('SPAN',(0, 0), (-1, 0)),
        ('ALIGN', (0, 1), (-1, -1), 'LEFT'), 
        ('VALIGN', (0, 1), (-1, -1), 'TOP'),
        ('SPAN',(0, 0), (-1, 0)),
    ]))
    story.append(oportunidad_table)
    story.append(Spacer(1, 0.2*inch))


    try:
        doc.build(story)
        print(f"Ficha generada en: {output_path}")
    except Exception as e:
        print(f"Error generating PDF with ReportLab: {e}")